In [ ]:
import sys
sys.path.append('env/lib/python3.13/site-packages')

In [ ]:
import numpy as np
import scipy as sp
import cvxpy as cp
import cvxportfolio as cvx
import matplotlib.pyplot as plt

In [ ]:
sample_returns = np.load('sample_returns.npy')[::1]

In [ ]:
# sample_returns = cvx.YahooFinance('^VIX').data['open'].dropna().values

In [ ]:
sample_returns = cvx.YahooFinance('AAPL').data['return'].dropna().values

In [ ]:
# np.sort(sample_returns)

In [ ]:
# sample_returns = (np.sin(np.random.uniform(0, 2*np.pi, 10000)))

In [ ]:
(qnorm, q), _ = sp.stats.probplot(sample_returns, dist="norm", plot=plt)

In [ ]:
plt.plot(qnorm, q, 'o')

In [ ]:
nodes = np.linspace(-4,4,8+1)
# nodes[0] = -100
# nodes[-1] = 100
nodes

In [ ]:
# nodes = np.quantile(qnorm, np.linspace(0,1, 1001))
# nodes

In [ ]:
for i in range(len(nodes)-1):
    mask = (qnorm<=nodes[i+1])&(qnorm>nodes[i])
    plt.plot(qnorm[mask], q[mask], 'o')

In [ ]:
a_s = cp.Variable(len(nodes)-1)
b_s = cp.Variable(len(nodes)-1)

In [ ]:
LAMBDA_REG = 1.

objective = 0.
for i in range(len(nodes)-1):
    mask = (qnorm<=nodes[i+1])&(qnorm>nodes[i])
    if np.sum(mask) == 0:
        continue
    my_qnorm = qnorm[mask]
    my_q = q[mask]
    objective += cp.sum_squares(a_s[i] * my_qnorm + b_s[i] - my_q)

objective += LAMBDA_REG * cp.sum_squares(cp.diff(a_s))
constraints = [
    a_s[i] * nodes[i+1] + b_s[i] == a_s[i+1] * nodes[i+1] + b_s[i+1]
    for i in range(len(nodes) - 2)
]
constraints += [a_s >= 1e-8]

cp.Problem(
    cp.Minimize(objective),
    constraints
).solve()

In [ ]:
plt.plot(a_s.value)

In [ ]:
# (sample_returns==0.).mean()

In [ ]:
for i in range(len(nodes)-1):
    mask = (qnorm<=nodes[i+1])&(qnorm>nodes[i])
    plt.plot(qnorm[mask], qnorm[mask] * a_s.value[i] + b_s.value[i])
    plt.plot(qnorm[mask], q[mask], 'o', alpha=.01)

In [ ]:
my_node_values = [
    a_s.value[i] * nodes[i] + b_s.value[i]
    for i in range(len(nodes) - 1)
]
my_node_values.append(a_s.value[-1] * nodes[-1] + b_s.value[-1])
my_node_values = np.array(my_node_values)
my_node_values

In [ ]:
transformed = np.interp(sample_returns, my_node_values, nodes )

In [ ]:
_ = sp.stats.probplot(transformed, dist="norm", plot=plt)

### For comparison, old approach 

In [ ]:
N_QUANTILES = 100
_qnorm_extrapolate = 100


def compute_normal_quantiles(nobs: int) -> np.array:
    """Compute normal quantiles for non-duplicate inputs."""
    p = (0.5 + np.arange(float(nobs))) / float(nobs)
    return sp.stats.norm.ppf(p)

_quantiles = np.linspace(0., 1., N_QUANTILES + 1)[1:-1]
qs = np.quantile(sample_returns, _quantiles)

_raw_qnorm = compute_normal_quantiles(len(sample_returns))
q_norm_orig = np.quantile(_raw_qnorm, _quantiles)
q_norm = np.copy(q_norm_orig)

q_norm[0] = min(-_qnorm_extrapolate, q_norm[0])
q_norm[-1] = max(_qnorm_extrapolate, q_norm[-1])


# extrapolation
slope_left = (qs[1] - qs[0]) / (
    q_norm_orig[1] - q_norm_orig[0])
slope_right = (qs[-1] - qs[-2]) / (
    q_norm_orig[-1] - q_norm_orig[-2])

qs[0] = qs[1] - slope_left * (q_norm[1] - q_norm[0])
qs[-1] = qs[-2] + slope_right * (q_norm[-1] - q_norm[-2])

In [ ]:
# q_norm

In [ ]:
# qs

In [ ]:
transformed2 = np.interp(sample_returns, qs, q_norm )

In [ ]:
_ = sp.stats.probplot(transformed2, dist="norm", plot=plt)